## Try 3

Utilizamos GridSearchCV con RandomForestClassifier. Las transformaciones son las mismas que la del baseline.

In [1]:
# Import the required packages
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
train_df = pd.read_csv("../data/travel_insurance_prediction_train.csv")
test_df = pd.read_csv("../data/travel_insurance_prediction_test.csv")

In [6]:
from sklearn.compose import make_column_transformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV, train_test_split, RandomizedSearchCV
from sklearn.preprocessing import KBinsDiscretizer, OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from sklearn.neighbors import KNeighborsClassifier

In [7]:
transformer = make_column_transformer(
    (KBinsDiscretizer(n_bins=5, encode="ordinal", strategy="quantile"), ["Age"]),
    (OneHotEncoder(categories="auto", dtype="int", handle_unknown="ignore"),
     ["Employment Type", "GraduateOrNot", "FamilyMembers", "FrequentFlyer", "EverTravelledAbroad"]),
    (MinMaxScaler(),["AnnualIncome"]),
    remainder="passthrough")

In [8]:
X = transformer.fit_transform(train_df.drop(columns=["Customer", "TravelInsurance"]))
y = train_df["TravelInsurance"].values
X_test_transform = transformer.transform(test_df.drop(columns=["Customer"]))

In [20]:
rfc = RandomForestClassifier()
param = {
    'n_estimators':[*range(100,300,25)],
    'criterion':['gini'],
    'max_depth':[25,20,None,10],
    'min_samples_split':[2,5,None,15,20]
}
cv = GridSearchCV(estimator=rfc,param_grid=param, scoring='f1',n_jobs=-1,cv=5)
cv.fit(X, y)
from sklearn.metrics import classification_report
print(classification_report(y, cv.predict(X)))
best_tree_clf = cv.best_estimator_

/home/exequiel/anaconda3/envs/Diplomatura2021/lib/python3.8/site-packages/sklearn/model_selection/_search.py:922: UserWarning: One or more of the test scores are non-finite: [0.69083581 0.68404354 0.69024645 0.68800289 0.68946221 0.6825614
 0.68122803 0.68828413 0.70586802 0.71335548 0.71142217 0.71760922
 0.71256739 0.71314782 0.71664552 0.71223423        nan        nan
        nan        nan        nan        nan        nan        nan
 0.7126338  0.71117347 0.71339101 0.70807739 0.70384445 0.70883
 0.71331843 0.7067267  0.70436366 0.71117061 0.70590366 0.70069359
 0.70530723 0.7104391  0.70588455 0.70074044 0.68628511 0.68689673
 0.68618371 0.6869821  0.69515916 0.6839288  0.68554286 0.68972316
 0.71364699 0.71546662 0.70664119 0.71275292 0.71428019 0.71920138
 0.71869444 0.71949172        nan        nan        nan        nan
        nan        nan        nan        nan 0.71095577 0.709016
 0.71122511 0.7141386  0.71270198 0.71018222 0.7054677  0.71417456
 0.6990628  0.70072809 0.702

              precision    recall  f1-score   support

           0       0.87      0.96      0.91       958
           1       0.91      0.74      0.82       532

    accuracy                           0.88      1490
   macro avg       0.89      0.85      0.87      1490
weighted avg       0.89      0.88      0.88      1490



In [23]:
test_id = test_df["Customer"]
test_pred = best_tree_clf.predict(X_test_transform)

In [24]:
submission = pd.DataFrame(list(zip(test_id, test_pred)), columns=["Customer", "TravelInsurance"])

In [25]:
submission.to_csv("travel_insurance_submission-try-C.csv", header=True, index=False)

## Observación

Realizamos mas pruebas para la competencia con diferentes algoritmos, solamente mostramos las notebooks que mejor resultados nos dieron en la competencia de kaggle